In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import time
import re
import os
import sqlite3

import requests
import yfinance as yf

import string
import random
import unidecode
from bs4 import BeautifulSoup

import settings

### DADOAS ABEERTOS DOWNLOAD

In [111]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd

In [122]:
# Base URL and report types
base_url = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/"
reports = ["ITR", "DFP"]

data_folder = r"D:\Fausto Stangler\Documentos\Python\FLY\backend\data"

In [112]:
# Container for the extracted data
zip_files = []

# Iterate over report types and collect .zip file URLs
for report_type in reports:
    url = f"{base_url}{report_type}/DADOS/"
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request is successful
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract all .zip links
    zip_files.extend(
        [
            url + a["href"]
            for a in soup.find_all("a", href=True)
            if a["href"].endswith(".zip")
        ]
    )

In [180]:
level_skip = "no_level"
keywords = ["ind", "con"]

dataframes = {}

for zip_url in zip_files:
    print(f"{zip_url}")
    try:
        # Determine a cache file path based on the URL
        cached_file_path = os.path.join(data_folder, os.path.basename(zip_url))

        # Check if the file is already cached
        if os.path.exists(cached_file_path):
            with open(cached_file_path, "rb") as cached_file:
                content = cached_file.read()
        else:
            print(f"Downloading: {zip_url}")
            # Request the .zip file
            response = requests.get(zip_url)
            response.raise_for_status()

            # Save to cache after validation
            content = response.content
            with open(cached_file_path, "wb") as cached_file:
                cached_file.write(content)

        # Open the .zip file
        with zipfile.ZipFile(io.BytesIO(content)) as z:
            # List all files in the ZIP archive
            file_names = z.namelist()
            for file_name in file_names:
                try:
                    # Check if the file is a CSV or another readable format
                    if file_name.endswith(".csv") and any(
                        keyword in file_name for keyword in keywords
                    ):
                        # Parse the filename for year, level, and type
                        parts = file_name.split("_")
                        year = parts[-1].split(".")[0]  # Extract the year
                        level = parts[-2] if parts[-2] in ["con", "ind"] else level_skip
                        doc_type = parts[-3] if level in ["con", "ind"] else parts[-2]

                        # Construct the path to the file
                        file_path = os.path.join(data_folder, file_name)
                        print(f"{file_path}")

                        # Read the CSV into a DataFrame
                        with z.open(file_name) as f:
                            # df = pd.read_csv(f, encoding='latin1', sep=';', error_bad_lines=False)
                            df = pd.read_csv(
                                f, encoding="latin1", sep=";"
                            )  # Read CSV file

                            # Initialize nested dictionaries as needed
                            if year not in dataframes:
                                dataframes[year] = {}
                            if doc_type not in dataframes[year]:
                                dataframes[year][doc_type] = {}

                            # Store the DataFrame in the appropriate place
                            if level != level_skip:
                                dataframes[year][doc_type][level] = df

                except Exception as e:
                    print(f"Failed to process {file_name}: {e}")

        break
    except Exception as e:
        print(f"Error processing {zip_url}: {e}")
        break

https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2011.zip
D:\Fausto Stangler\Documentos\Python\FLY\backend\data\itr_cia_aberta_BPA_con_2011.csv
D:\Fausto Stangler\Documentos\Python\FLY\backend\data\itr_cia_aberta_BPA_ind_2011.csv
D:\Fausto Stangler\Documentos\Python\FLY\backend\data\itr_cia_aberta_BPP_con_2011.csv
D:\Fausto Stangler\Documentos\Python\FLY\backend\data\itr_cia_aberta_BPP_ind_2011.csv
D:\Fausto Stangler\Documentos\Python\FLY\backend\data\itr_cia_aberta_DFC_MD_con_2011.csv
D:\Fausto Stangler\Documentos\Python\FLY\backend\data\itr_cia_aberta_DFC_MD_ind_2011.csv
D:\Fausto Stangler\Documentos\Python\FLY\backend\data\itr_cia_aberta_DFC_MI_con_2011.csv
D:\Fausto Stangler\Documentos\Python\FLY\backend\data\itr_cia_aberta_DFC_MI_ind_2011.csv
D:\Fausto Stangler\Documentos\Python\FLY\backend\data\itr_cia_aberta_DMPL_con_2011.csv
D:\Fausto Stangler\Documentos\Python\FLY\backend\data\itr_cia_aberta_DMPL_ind_2011.csv
D:\Fausto Stangler\Documentos\Python\FLY\backe

In [183]:
dataframes["2011"].keys()

dict_keys(['BPA', 'BPP', 'MD', 'MI', 'DMPL', 'DRA', 'DRE', 'DVA'])

In [ ]:
existing_df

In [184]:
df = dataframes["2011"]["BPA"]["con"]

In [187]:
df = df[df["ORDEM_EXERC"] != "PENÚLTIMO"]
df

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2011-03-31,1,Ativo Total,149751700.0,S
3,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2011-03-31,1.01,Ativo Circulante,34736318.0,S
5,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2011-03-31,1.01.01,Caixa e Equivalentes de Caixa,12358365.0,S
7,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2011-03-31,1.01.01.01,Caixa e equivalente de caixa,9763580.0,N
9,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2011-03-31,1.01.01.02,Caixa restrito,2594785.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135861,97.837.181/0001-47,2011-09-30,1,DEXCO S.A.,21091,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2011-09-30,1.02.04.01.03,"Softwares, marcas e patentes",28262.0,N
135863,97.837.181/0001-47,2011-09-30,1,DEXCO S.A.,21091,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2011-09-30,1.02.04.02,Goodwill,226819.0,S
135865,97.837.181/0001-47,2011-09-30,1,DEXCO S.A.,21091,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2011-09-30,1.02.04.02.01,Goodwill na associação da Satipel em 2009,187573.0,N
135867,97.837.181/0001-47,2011-09-30,1,DEXCO S.A.,21091,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2011-09-30,1.02.04.02.02,Goodwill na aquisição Cerâmica Monte Carlo em ...,22154.0,N


In [126]:
# download zip

In [127]:
# extract content

In [128]:
# import content as df

In [129]:
# process df to match statements columns

In [130]:
# save as some appropriate format, maybe check nsd numbers or something yet to be defined

### ENET DOWNLOAD

#### Get new NSD ENET list

In [131]:
# load all nsd and filter only ITR and DFP

In [132]:
# load all saved-reports-nsd and exclude saved-reports-nsd from all-nsd list

#### download enet nsd-new reports

In [133]:
# open nsd page, get info and generate enet download link

In [134]:
# download enet zip, open and read xls and delete zip and other files

#### process and clean xls data

#### incremental save nsd reports

In [135]:
# save reports

In [136]:
# update saved-reports-nsd list

### Financial Ratios

In [137]:
def duplicate_dfs_types(
    df,
    source_type="Dados da Empresa",
    target_types=["DFs Consolidadas", "DFs Individuais"],
):
    """
    Duplica condicionalmente as linhas de um tipo específico para outros tipos,
    baseando-se na existência prévia desses tipos para a mesma empresa e quarter.

    Parâmetros:
    - df (pd.DataFrame): DataFrame original contendo os dados financeiros.
    - source_type (str): O tipo de linha que será duplicado. Padrão: 'Dados da Empresa'.
    - target_types (list of str): Lista dos tipos para os quais as linhas serão duplicadas.
                                   Padrão: ['DFs Consolidadas', 'DFs Individuais'].

    Retorna:
    - pd.DataFrame: DataFrame atualizado com as duplicações condicionais.
    """

    # Passo 1: Identificar combinações existentes de company_name e quarter para cada target_type
    existing_combinations = {}
    for target in target_types:
        existing_keys = df[df["type"] == target][
            ["company_name", "quarter"]
        ].drop_duplicates()
        existing_combinations[target] = existing_keys

    # Passo 2: Filtrar as linhas do source_type
    source_df = df[df["type"] == source_type].copy()

    # Passo 3: Para cada target_type, verificar onde duplicar
    duplicated_dfs = []
    for target in target_types:
        # Obter as combinações onde já existe o target_type
        target_keys = existing_combinations[target]

        # Realizar um merge para identificar quais linhas do source_df têm a combinação existente
        to_duplicate = source_df.merge(
            target_keys,
            on=["company_name", "quarter"],
            how="inner",
            suffixes=("", "_target"),
        )

        if not to_duplicate.empty:
            # Duplicar as linhas e alterar o type para o target_type
            duplicated = to_duplicate.copy()
            duplicated["type"] = target
            duplicated_dfs.append(duplicated)
            # print(f"Duplicando {len(duplicated)} linhas para o tipo '{target}'.")
        else:
            # print(f"Nenhuma duplicação necessária para o tipo '{target}'.")
            pass

    # Passo 4: Concatenar todas as duplicações
    if duplicated_dfs:
        duplicated_df = pd.concat(duplicated_dfs, ignore_index=True)
    else:
        duplicated_df = pd.DataFrame(columns=df.columns)
        # print("Nenhuma duplicação realizada.")

    # Passo 5: Remover as linhas originais do source_type
    df_filtered = df[df["type"] != source_type].copy()

    # Passo 6: Adicionar as duplicações ao DataFrame
    if not duplicated_df.empty:
        df_updated = pd.concat([df_filtered, duplicated_df], ignore_index=True)
    else:
        df_updated = df_filtered.copy()

    # Passo 7: Resetar o índice e ordenar o DataFrame (opcional)
    df_updated.reset_index(drop=True, inplace=True)

    # Passo 8: Ordenar o DataFrame
    df_updated = df_updated.sort_values(
        by=[
            "sector",
            "subsector",
            "segment",
            "company_name",
            "quarter",
            "version",
            "type",
            "account",
            "description",
        ]
    ).reset_index(drop=True)

    return df_updated

In [138]:
def add_indicators(df, frame_name, indicator_list):
    """
    Calcula indicadores financeiros e os adiciona como novas linhas no DataFrame.

    Parâmetros:
    - df (pd.DataFrame): DataFrame original contendo os dados financeiros.
    - indicator_list (list of dict): Lista de dicionários com definições dos indicadores.

    Retorna:
    - pd.DataFrame: DataFrame atualizado com as novas linhas de indicadores.
    """

    # Função para realizar divisão segura
    def safe_division(numerator, denominator):
        with np.errstate(divide="ignore", invalid="ignore"):
            result = numerator / denominator
            result = result.replace(
                [np.inf, -np.inf], np.nan
            )  # Substitui infinitos por NaN
            return result

    # Passo 1: Pivotar o DataFrame com contas como colunas e incluir 'quarter' no índice
    pivot_df = df.pivot_table(
        index=["company_name", "type", "quarter"],
        columns="account",
        values="value",
        aggfunc="sum",
        fill_value=0,  # Substitui valores ausentes por 0
    ).reset_index()

    # print("Colunas do DataFrame Pivotado:", pivot_df.columns.tolist())

    # Passo 2: Calcular os Indicadores
    for indicator in indicator_list:
        column_name = indicator["description"]
        column_account = indicator["account"]
        try:
            # Aplicar a fórmula para calcular o indicador
            pivot_df[column_name] = indicator["formula"](pivot_df)
            # print(f"Indicador '{column_name}' calculado com sucesso.")
        except KeyError as e:
            print(f"'{column_account} - {column_name}': a conta {e} não existe")
            pivot_df[column_name] = np.nan  # Atribuir NaN se a coluna não existir
        except Exception as e:
            print(
                f"{column_name}': {e} Erro ao calcular o indicador '{column_name}': {e}"
            )
            pivot_df[column_name] = np.nan  # Atribuir NaN em caso de erro

    # Passo 3: Criar Novas Linhas para os Indicadores
    new_rows = []

    for indicator in indicator_list:
        account = indicator["account"]
        description = indicator["description"]

        if description not in pivot_df.columns:
            print(
                f"Aviso: Indicador '{description}' não foi calculado e será ignorado."
            )
            continue

        # Extrair os valores calculados do indicador
        pivot_df["value"] = pivot_df[description]

        # Selecionar as colunas relevantes
        indicator_values = pivot_df[["company_name", "type", "quarter", "value"]].copy()
        indicator_values["account"] = account
        indicator_values["description"] = description

        # Atribuir 'frame' como 'Indicadores' para identificar facilmente as novas linhas
        indicator_values["frame"] = frame_name

        # Preencher as colunas faltantes com informações do DataFrame original
        # Selecionar combinações únicas de 'company_name', 'type' e 'quarter'
        metadata = df[
            [
                "company_name",
                "type",
                "quarter",
                "nsd",
                "sector",
                "subsector",
                "segment",
                "version",
            ]
        ].drop_duplicates(subset=["company_name", "type", "quarter"], keep="first")

        # Mesclar com 'metadata' para preencher 'nsd', 'sector', etc.
        indicator_row = indicator_values.merge(
            metadata, on=["company_name", "type", "quarter"], how="left"
        )

        # Reordenar as colunas para corresponder ao DataFrame original
        indicator_row = indicator_row[
            [
                "nsd",
                "sector",
                "subsector",
                "segment",
                "company_name",
                "quarter",
                "version",
                "type",
                "frame",
                "account",
                "description",
                "value",
            ]
        ]

        # Adicionar à lista de novas linhas
        new_rows.append(indicator_row)

    # Combinar todas as novas linhas de indicadores
    if new_rows:
        new_rows_df = pd.concat(new_rows, ignore_index=True)
    else:
        new_rows_df = pd.DataFrame(columns=df.columns)
        # print("Nenhum indicador foi adicionado.")

    # Passo 4: Adicionar as Novas Linhas ao DataFrame Original
    updated_df = pd.concat([df, new_rows_df], ignore_index=True)

    # Passo 5: Tratamento Final dos Valores
    # Opcional: Preencher valores faltantes ou realizar outras limpezas
    # Exemplo: Substituir NaN por zero onde apropriado
    # updated_df['value'] = updated_df['value'].fillna(0)

    return updated_df

In [139]:
sector = "COMUNICACOES"
db_file = (
    f"..\data\{settings.db_filepath.split('.')[0]} {settings.statements_standard}.db"
)

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Fetch all table names excluding internal SQLite tables
cursor.execute(
    "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
)
tables = cursor.fetchall()

df = pd.read_sql_query(f"SELECT * FROM {sector}", conn)
# df['account_description'] = df['account'] + ' - ' + df['description']
df["quarter"] = pd.to_datetime(df["quarter"])

# df = clean_dados_da_empresa(df)
df = duplicate_dfs_types(df)

# Store unique values
uv = {}
for col in df.columns:
    if col not in ["nsd", "value"]:
        uv[col] = sorted(df[col].unique().tolist())

df[["account", "description"]].drop_duplicates().to_csv(
    "account-description.csv", index=False
)

NameError: name 'settings' is not defined

In [249]:
# Definition of indicator formulas
indicators_08 = [
    {
        "account": "08.01",
        "description": "ROE Return On Equity",
        "formula": lambda df: df["03.11"]
        / df["02.03"]
        * 10000,  # ROE = Net Profit / Shareholders' Equity * 100
    },
    {
        "account": "08.02",
        "description": "Índice de Liquidez Corrente",
        "formula": lambda df: df["01.01"]
        / df["02.01"],  # Current Ratio = Current Assets / Current Liabilities
    },
    # Add more indicators as needed
]

In [255]:
df = add_indicators(df, "Indicadores Fundamentalistas", indicators_08)
# Store unique values
uvi = {}
for col in df.columns:
    if col not in ["nsd", "value"]:
        uvi[col] = sorted(df[col].unique().tolist())

In [ ]:
mask = df["company_name"] == uvi["company_name"][7]
mask &= df["type"] == uvi["type"][1]
mask &= df["account"] == "08.02"
# mask &= df2['quarter'] == '2020-12-31'
df[mask]

In [ ]:
index_cols = [col for col in df2.columns if col not in ["quarter", "value"]]

# Verificar duplicatas
duplicatas = df2.duplicated(subset=index_cols + ["quarter"], keep=False)
num_duplicatas = duplicatas.sum()

if num_duplicatas > 0:
    print("Agregando duplicatas somando os valores.")
    df2_agregado = df2.groupby(index_cols + ["quarter"], as_index=False)["value"].sum()
    # df2_agregado = df2.groupby(index_cols + ['quarter'], as_index=False)['value'].mean() # opcionalmente calcular a média

else:
    df2_agregado = df2.copy()

# Pivotar o DataFrame
df_pivot = df2_agregado.pivot_table(
    index=index_cols,
    columns="quarter",
    values="value",
    aggfunc="first",  # 'first' assume que não há duplicatas após a agregação
).reset_index()

# # Obter uma lista dos quarters únicos, ordenados de forma decrescente
# sorted_quarters = sorted(df_pivot.columns[ len(index_cols): ], reverse=True)

# # Reordenar as colunas: primeiro os índices, depois os quarters em ordem decrescente
# df_pivot = df_pivot[index_cols + sorted_quarters]

# # Opcional: formatar os nomes das colunas de quarter para strings legíveis
# df_pivot.columns = list(index_cols) + [q.strftime('%Y-%m-%d') for q in sorted_quarters]

df_pivot.fillna(0, inplace=True)
df_pivot

In [ ]:
df_pivot.iloc[0:1,]

### Plots

In [8]:
sector = "COMUNICACOES"
db_file = (
    f"..\data\{settings.db_filepath.split('.')[0]} {settings.statements_standard}.db"
)

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Fetch all table names excluding internal SQLite tables
cursor.execute(
    "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
)
tables = cursor.fetchall()

df = pd.read_sql_query(f"SELECT * FROM {sector}", conn)
# df['account_description'] = df['account'] + ' - ' + df['description']
df["quarter"] = pd.to_datetime(df["quarter"])

In [10]:
# Store unique values
uv = {}
for col in df.columns:
    if col not in ["nsd", "value"]:
        uv[col] = sorted(df[col].unique().tolist())
uv

{'sector': ['COMUNICACOES'],
 'subsector': ['MIDIA', 'TELECOMUNICACOES'],
 'segment': ['PUBLICIDADE E PROPAGANDA', 'TELECOMUNICACOES'],
 'company_name': ['ALGAR TELECOM SA',
  'BRISANET PARTICIPACOES SA',
  'DESKTOP SA',
  'ELETROMIDIA SA',
  'GIGA MAIS FIBRA TELECOMUNICACOES SA',
  'OI SA',
  'TELEC BRASILEIRAS SA TELEBRAS',
  'TELEFONICA BRASIL SA',
  'TIM SA',
  'UNIFIQUE TELECOMUNICACOES SA'],
 'quarter': [Timestamp('2010-12-31 00:00:00'),
  Timestamp('2011-03-31 00:00:00'),
  Timestamp('2011-06-30 00:00:00'),
  Timestamp('2011-09-30 00:00:00'),
  Timestamp('2011-12-31 00:00:00'),
  Timestamp('2012-03-31 00:00:00'),
  Timestamp('2012-06-30 00:00:00'),
  Timestamp('2012-09-30 00:00:00'),
  Timestamp('2012-12-31 00:00:00'),
  Timestamp('2013-03-31 00:00:00'),
  Timestamp('2013-06-30 00:00:00'),
  Timestamp('2013-09-30 00:00:00'),
  Timestamp('2013-12-31 00:00:00'),
  Timestamp('2014-03-31 00:00:00'),
  Timestamp('2014-06-30 00:00:00'),
  Timestamp('2014-09-30 00:00:00'),
  Timestamp(

In [11]:
def plot_series(
    company_name,
    account,
    start_date=None,
    end_date=None,
    window=4,
    std_dev=2,
    normalize=True,
):
    """
    Plot a time series of a financial metric for a given company, with Bollinger Bands and custom blue color scheme.

    Args:
    - company_name: Name of the company.
    - account: The financial account to be plotted.
    - start_date: Run date for filtering the data (optional).
    - end_date: End date for filtering the data (optional).
    - window: The window size for the moving average (default is 4).
    - std_dev: The number of standard deviations for the Bollinger Bands (default is 2).
    """
    # Define blue color scheme
    color_blue = {
        "main": "blue",
        "mma": "lightblue",
        "bands": "lightblue",
        "fill": "rgba(173, 216, 230, 0.2)",
        "extra_bands": "lightgray",
        "extra_fill": "rgba(200, 200, 200, 0.2)",
    }

    # Try 'DFs Consolidadas' first
    if (df["type"] == "DFs Consolidadas").any():
        df_type = "DFs Consolidadas"
    else:
        df_type = "DFs Individuais"

    # Filter data
    data = df[
        (df["company_name"] == company_name)
        & (df["account"] == account)
        & (df["type"] == df_type)
    ]

    # Apply date filters if provided
    if start_date:
        data = data[data["quarter"] >= pd.to_datetime(start_date)]
    if end_date:
        data = data[data["quarter"] <= pd.to_datetime(end_date)]

    if data.empty:
        print("No data available for the given filters, check parameters.")
        return

    if normalize == True:
        # Drop the first row if its value is zero
        if data["value"].iloc[0] == 0:
            data = data.iloc[1:].copy()

        # Get the first non-zero value
        first_non_zero = data["value"].iloc[0]

        # Normalize the values by dividing by the first non-zero value
        data["value"] = data["value"] / first_non_zero

        # Scale the values so that the maximum is 100
        max_value = data["value"].max()
        data["value"] = (data["value"] / max_value) * 100

    # Sort data by date
    data = data.sort_values("quarter")

    # Calculate the moving average (Middle Band)
    data["mma"] = data["value"].rolling(window=window).mean()

    # Calculate the rolling standard deviation
    data["std_dev"] = data["value"].rolling(window=window).std()

    # Calculate upper and lower Bollinger Bands with std_dev multiplier
    data["upper_band"] = data["mma"] + std_dev * data["std_dev"]
    data["lower_band"] = data["mma"] - std_dev * data["std_dev"]

    # Calculate extra upper and lower bands using std_dev * 1.5
    extra_std_dev = std_dev * 1.5
    data["upper_band_extra"] = data["mma"] + extra_std_dev * data["std_dev"]
    data["lower_band_extra"] = data["mma"] - extra_std_dev * data["std_dev"]

    # Set description to the first non-null description value in the filtered data
    description = (
        data["description"].dropna().iloc[0]
        if "description" in data.columns and not data["description"].isna().all()
        else account
    )

    # Create line plot with original values
    fig = px.line(
        data,
        x="quarter",
        y="value",
        title=f"{account} - {description} with Bollinger Bands for {company_name} ({df_type})",
        labels={"quarter": "Quarter", "value": description},
        template="plotly_white",
    )

    # Add main line for the data
    fig.update_traces(
        line=dict(color=color_blue["main"]), name="Original Data", showlegend=False
    )

    # Add moving average (MMA) as a dashed light blue line
    fig.add_scatter(
        x=data["quarter"],
        y=data["mma"],
        mode="lines",
        name=f"Middle Band (window={window})",
        line=dict(color=color_blue["mma"], dash="dash"),
        showlegend=False,
    )

    # Add upper and lower Bollinger Bands (std_dev * 2) and group them
    fig.add_scatter(
        x=data["quarter"],
        y=data["upper_band"],
        mode="lines",
        name="Upper Band (std_dev * 2)",
        line=dict(dash="solid", color=color_blue["bands"]),
        legendgroup="Bands (std_dev * 2)",
        showlegend=False,
    )

    fig.add_scatter(
        x=data["quarter"],
        y=data["lower_band"],
        mode="lines",
        name="Lower Band (std_dev * 2)",
        line=dict(dash="solid", color=color_blue["bands"]),
        legendgroup="Bands (std_dev * 2)",
        showlegend=False,
    )

    # Fill the area between the upper and lower bands (std_dev * 2)
    fig.add_scatter(
        x=data["quarter"],
        y=data["upper_band"],
        fill=None,
        mode="lines",
        name="Upper Band Fill (std_dev * 2)",
        line=dict(color="rgba(0,0,0,0)"),
        showlegend=False,
        legendgroup="Bands (std_dev * 2)",
    )

    fig.add_scatter(
        x=data["quarter"],
        y=data["lower_band"],
        fill="tonexty",
        mode="lines",
        name="Lower Band Fill (std_dev * 2)",
        line=dict(color="rgba(0,0,0,0)"),
        fillcolor=color_blue["fill"],
        showlegend=False,
        legendgroup="Bands (std_dev * 2)",
    )

    # Add extra upper and lower Bollinger Bands (std_dev * 1.5) and group them
    fig.add_scatter(
        x=data["quarter"],
        y=data["upper_band_extra"],
        mode="lines",
        name="Upper Band (std_dev * 1.5)",
        line=dict(dash="dot", color=color_blue["extra_bands"]),
        legendgroup="Bands (std_dev * 1.5)",
        showlegend=False,
    )

    fig.add_scatter(
        x=data["quarter"],
        y=data["lower_band_extra"],
        mode="lines",
        name="Lower Band (std_dev * 1.5)",
        line=dict(dash="dot", color=color_blue["extra_bands"]),
        legendgroup="Bands (std_dev * 1.5)",
        showlegend=False,
    )

    # Fill the area between the extra upper and lower bands (std_dev * 1.5)
    fig.add_scatter(
        x=data["quarter"],
        y=data["upper_band_extra"],
        fill=None,
        mode="lines",
        name="Upper Band Extra Fill (std_dev * 1.5)",
        line=dict(color="rgba(0,0,0,0)"),
        showlegend=False,
        legendgroup="Bands (std_dev * 1.5)",
    )

    fig.add_scatter(
        x=data["quarter"],
        y=data["lower_band_extra"],
        fill="tonexty",
        mode="lines",
        name="Lower Band Extra Fill (std_dev * 1.5)",
        line=dict(color="rgba(0,0,0,0)"),
        fillcolor=color_blue["extra_fill"],
        showlegend=False,
        legendgroup="Bands (std_dev * 1.5)",
    )

    # Update layout
    fig.update_layout(xaxis_title="Quarter", yaxis_title=description)

    fig.show()

    return data

In [ ]:
data = plot_series(
    company_name=uv["company_name"][7],  # Change to your desired company
    account="01.01",
    # start_date='2015-01-01',
    # end_date='2023-06-30',
    window=4,
    normalize=False,
)

In [119]:
def plot_time_series(
    company_name, accounts, start_date=None, end_date=None, plot_type="stacked"
):
    """
    Plot a stacked, overlapping, line-only, 100% stacked, or normalized stacked area chart of financial metrics for multiple accounts of a given company.

    Args:
    - company_name: Name of the company.
    - accounts: List of financial accounts to be plotted.
    - start_date: Run date for filtering the data (optional).
    - end_date: End date for filtering the data (optional).
    - plot_type: Determines the type of plot: "stacked", "overlapping", "lines", "100_stacked", or "normalized_stacked".
    """
    # Filter and combine data for all accounts
    filtered_data = pd.DataFrame()

    for account in accounts:
        # Try 'DFs Consolidadas' first
        if (df["type"] == "DFs Consolidadas").any():
            df_type = "DFs Consolidadas"
        else:
            df_type = "DFs Individuais"

        # Filter data for the current account
        data = df[
            (df["company_name"] == company_name)
            & (df["account"] == account)
            & (df["type"] == df_type)
        ]

        # Apply date filters if provided
        if start_date:
            data = data[data["quarter"] >= pd.to_datetime(start_date)]
        if end_date:
            data = data[data["quarter"] <= pd.to_datetime(end_date)]

        if data.empty:
            print(f"No data available for account {account}. Skipping.")
            continue

        # Apply normalization only when plot_type is "normalized_stacked"
        if plot_type == "normalized_stacked":
            # Drop the first row if its value is zero
            if data["value"].iloc[0] == 0:
                data = data.iloc[1:].copy()

            # Get the first non-zero value and normalize all data by that value
            first_non_zero_value = data["value"][data["value"] != 0].iloc[0]

            # Normalize to ensure the first non-zero value is 1
            data.loc[:, "value"] = data["value"] / first_non_zero_value

        # Sort data by date
        data = data.sort_values("quarter")

        # Add the account as a column for identifying each account
        data["account"] = account

        # Append to the cumulative data
        filtered_data = pd.concat([filtered_data, data])

    if filtered_data.empty:
        print("No data available for the given filters, check parameters.")
        return

    # Determine the Y-axis label
    y_axis_title = "Value"

    # Plot logic for different plot types
    if plot_type == "stacked":
        # Stacked area plot with raw values
        fig = px.area(
            filtered_data,
            x="quarter",
            y="value",
            color="account",
            title=f"Stacked Area Plot of Accounts for {company_name}",
            labels={"quarter": "Quarter", "value": y_axis_title},
            template="plotly_white",
        )

    elif plot_type == "overlapping":
        # Overlapping area plot (using filled line plots to avoid stacking)
        fig = px.line(
            filtered_data,
            x="quarter",
            y="value",
            color="account",
            title=f"Overlapping Area Plot of Accounts for {company_name}",
            labels={"quarter": "Quarter", "value": y_axis_title},
            template="plotly_white",
        )

        # Add filled areas to create the overlapping effect
        for account in accounts:
            account_data = filtered_data[filtered_data["account"] == account]
            fig.add_scatter(
                x=account_data["quarter"],
                y=account_data["value"],
                mode="lines",
                line=dict(width=0.5),
                fill="tozeroy",
                name=account,
                legendgroup=account,  # Group line and area under the same legend
                showlegend=False,  # Hide the second entry for the filled area
            )

    elif plot_type == "lines":
        # Line-only plot with no filling
        fig = px.line(
            filtered_data,
            x="quarter",
            y="value",
            color="account",
            title=f"Line-Only Plot of Accounts for {company_name}",
            labels={"quarter": "Quarter", "value": y_axis_title},
            template="plotly_white",
        )

    elif plot_type == "100_stacked":
        # 100% stacked area plot (scaled to show percentages)
        filtered_data["total_per_quarter"] = filtered_data.groupby("quarter")[
            "value"
        ].transform("sum")
        filtered_data["value"] = (
            filtered_data["value"] / filtered_data["total_per_quarter"]
        ) * 100
        y_axis_title = "Percentage Contribution (%)"

        fig = px.area(
            filtered_data,
            x="quarter",
            y="value",
            color="account",
            title=f"100% Stacked Area Plot of Accounts for {company_name}",
            labels={"quarter": "Quarter", "value": y_axis_title},
            template="plotly_white",
        )

    elif plot_type == "normalized_stacked":
        # Normalized stacked area plot (no percentage, just normalized values for each account)
        y_axis_title = "Normalized Value"

        fig = px.area(
            filtered_data,
            x="quarter",
            y="value",
            color="account",
            title=f"Normalized Stacked Area Plot of Accounts for {company_name}",
            labels={"quarter": "Quarter", "value": y_axis_title},
            template="plotly_white",
        )

    # Update layout
    fig.update_layout(xaxis_title="Quarter", yaxis_title=y_axis_title, showlegend=True)

    fig.show()

    return filtered_data

In [ ]:
#     - plot_type: Determines the type of plot: "stacked", "lines", "overlapping", "100_stacked", or "normalized_stacked".
# Example usage: Line-only plot
data = plot_time_series(
    company_name=uv["company_name"][7],
    accounts=[
        "01.01.01",
        "01.01.02",
        "01.01.03",
        "01.01.04",
    ],
    plot_type="stacked",  # Line-only plot
    start_date="2013-12-31",
)

# Example usage: Line-only plot
data = plot_time_series(
    company_name=uv["company_name"][7],
    accounts=[
        "01.01.01",
        "01.01.02",
        "01.01.03",
        "01.01.04",
    ],
    plot_type="100_stacked",  # Line-only plot
    start_date="2013-12-31",
)

In [ ]:
def plot_time_series(
    company_name, accounts, start_date=None, end_date=None, plot_type="stacked"
):
    """
    Plot a stacked, overlapping, line-only, 100% stacked, or normalized stacked area chart of financial metrics for multiple accounts of a given company.

    Args:
    - company_name: Name of the company.
    - accounts: List of financial accounts to be plotted.
    - start_date: Run date for filtering the data (optional).
    - end_date: End date for filtering the data (optional).
    - plot_type: Determines the type of plot: "stacked", "overlapping", "lines", "100_stacked", or "normalized_stacked".
    """
    # Filter and combine data for all accounts
    filtered_data = pd.DataFrame()

    for account in accounts:
        # Try 'DFs Consolidadas' first
        if (df["type"] == "DFs Consolidadas").any():
            df_type = "DFs Consolidadas"
        else:
            df_type = "DFs Individuais"

        # Filter data for the current account
        data = df[
            (df["company_name"] == company_name)
            & (df["account"] == account)
            & (df["type"] == df_type)
        ]

        # Apply date filters if provided
        if start_date:
            data = data[data["quarter"] >= pd.to_datetime(start_date)]
        if end_date:
            data = data[data["quarter"] <= pd.to_datetime(end_date)]

        if data.empty:
            print(f"No data available for account {account}. Skipping.")
            continue

        # Apply normalization only when plot_type is "normalized_stacked"
        if plot_type == "normalized_stacked":
            # Drop the first row if its value is zero
            if data["value"].iloc[0] == 0:
                data = data.iloc[1:].copy()

            # Get the first non-zero value and normalize all data by that value
            first_non_zero_value = data["value"][data["value"] != 0].iloc[0]

            # Normalize to ensure the first non-zero value is 1
            data.loc[:, "value"] = data["value"] / first_non_zero_value

        # Sort data by date
        data = data.sort_values("quarter")

        # Add the account as a column for identifying each account
        data["account"] = account

        # Append to the cumulative data
        filtered_data = pd.concat([filtered_data, data])

    if filtered_data.empty:
        print("No data available for the given filters, check parameters.")
        return

    # Determine the Y-axis label
    y_axis_title = "Value"

    # Plot logic for different plot types
    if plot_type == "stacked":
        # Stacked area plot with raw values
        fig = px.area(
            filtered_data,
            x="quarter",
            y="value",
            color="account",
            title=f"Stacked Area Plot of Accounts for {company_name}",
            labels={"quarter": "Quarter", "value": y_axis_title},
            template="plotly_white",
        )

    elif plot_type == "overlapping":
        # Overlapping area plot (using filled line plots to avoid stacking)
        fig = px.line(
            filtered_data,
            x="quarter",
            y="value",
            color="account",
            title=f"Overlapping Area Plot of Accounts for {company_name}",
            labels={"quarter": "Quarter", "value": y_axis_title},
            template="plotly_white",
        )

        # Add filled areas to create the overlapping effect
        for account in accounts:
            account_data = filtered_data[filtered_data["account"] == account]
            fig.add_scatter(
                x=account_data["quarter"],
                y=account_data["value"],
                mode="lines",
                line=dict(width=0.5),
                fill="tozeroy",
                name=account,
                legendgroup=account,  # Group line and area under the same legend
                showlegend=False,  # Hide the second entry for the filled area
            )

    elif plot_type == "lines":
        # Line-only plot with no filling
        fig = px.line(
            filtered_data,
            x="quarter",
            y="value",
            color="account",
            title=f"Line-Only Plot of Accounts for {company_name}",
            labels={"quarter": "Quarter", "value": y_axis_title},
            template="plotly_white",
        )

    elif plot_type == "100_stacked":
        # 100% stacked area plot (scaled to show percentages)
        filtered_data["total_per_quarter"] = filtered_data.groupby("quarter")[
            "value"
        ].transform("sum")
        filtered_data["value"] = (
            filtered_data["value"] / filtered_data["total_per_quarter"]
        ) * 100
        y_axis_title = "Percentage Contribution (%)"

        fig = px.area(
            filtered_data,
            x="quarter",
            y="value",
            color="account",
            title=f"100% Stacked Area Plot of Accounts for {company_name}",
            labels={"quarter": "Quarter", "value": y_axis_title},
            template="plotly_white",
        )

    elif plot_type == "normalized_stacked":
        # Normalized stacked area plot (no percentage, just normalized values for each account)
        y_axis_title = "Normalized Value"

        fig = px.area(
            filtered_data,
            x="quarter",
            y="value",
            color="account",
            title=f"Normalized Stacked Area Plot of Accounts for {company_name}",
            labels={"quarter": "Quarter", "value": y_axis_title},
            template="plotly_white",
        )

    # Update layout
    fig.update_layout(xaxis_title="Quarter", yaxis_title=y_axis_title, showlegend=True)

    return fig  # Return the figure, not the DataFrame


from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Initialize a subplot figure with 2 rows and 1 column
fig = make_subplots(
    rows=2, cols=1, subplot_titles=("Stacked Plot", "100% Stacked Plot")
)

# Call plot_time_series for the stacked plot and add to the first subplot (row 1)
fig_stacked = plot_time_series(
    company_name=uv["company_name"][7],
    accounts=["01.01.01", "01.01.02", "01.01.03", "01.01.04"],
    plot_type="stacked",
    start_date="2013-12-31",
)

# Add the traces from the stacked plot to the first subplot
for trace in fig_stacked["data"]:
    fig.add_trace(trace, row=1, col=1)

# Call plot_time_series for the 100% stacked plot and add to the second subplot (row 2)
fig_100_stacked = plot_time_series(
    company_name=uv["company_name"][7],
    accounts=["01.01.01", "01.01.02", "01.01.03", "01.01.04"],
    plot_type="100_stacked",
    start_date="2013-12-31",
)

# Add the traces from the 100% stacked plot to the second subplot
for trace in fig_100_stacked["data"]:
    fig.add_trace(trace, row=2, col=1)

# Update the layout for the entire figure
fig.update_layout(
    title_text="Comparison of Stacked and 100% Stacked Plots",
    height=800,
    showlegend=False,  # Hide the legend since it might overlap
)

# Show the plot
fig.show()

In [220]:
import plotly.graph_objects as go
import pandas as pd


def plot_company_comparison(
    df, companies, account, start_date=None, end_date=None, plot_type="lines"
):
    """
    Compare a financial metric (account) across multiple companies, filtered by 'df_type',
    and display the average of the companies.

    Args:
    - df: DataFrame containing the financial data.
    - companies: List of company names to compare.
    - account: The financial account to be plotted (e.g., '03.11 - Lucro do Período').
    - start_date: Run date for filtering the data (optional).
    - end_date: End date for filtering the data (optional).
    - plot_type: Determines whether to plot 'lines' or 'normalized'.
    """
    # Try 'DFs Consolidadas' first
    if (df["type"] == "DFs Consolidadas").any():
        df_type = "DFs Consolidadas"
    else:
        df_type = "DFs Individuais"

    # Filter data for the specified companies, account, and df_type
    data = df[
        (df["company_name"].isin(companies))
        & (df["account"] == account)
        & (df["type"] == df_type)
    ].copy()  # Make a deep copy to avoid SettingWithCopyWarning

    # Apply start and end date filters if provided
    if start_date:
        data = data[
            data["quarter"] >= pd.to_datetime(start_date)
        ].copy()  # Copy after filtering
    if end_date:
        data = data[
            data["quarter"] <= pd.to_datetime(end_date)
        ].copy()  # Copy after filtering

    if data.empty:
        print("No data available for the given filters.")
        return

    # Get the description dynamically from the DataFrame based on the 'account' field
    description = (
        data["description"].dropna().iloc[0]
        if "description" in data.columns and not data["description"].isna().all()
        else account
    )

    # Create the figure
    fig = go.Figure()

    # Add line plot for each company (no markers)
    for company in companies:
        company_data = (
            data[data["company_name"] == company].sort_values("quarter").copy()
        )  # Avoid modifying the original DataFrame

        if not company_data.empty:
            if plot_type == "normalized":
                # Try to get the first non-zero value for normalization
                non_zero_values = company_data["value"][company_data["value"] != 0]

                if not non_zero_values.empty:
                    first_non_zero_value = non_zero_values.iloc[0]
                    # Normalize the values: scale from 1 to proportional max_value
                    company_data.loc[:, "normalized_value"] = (
                        company_data["value"] / first_non_zero_value
                    )
                else:
                    print(
                        f"Warning: No non-zero values for {company}. Skipping normalization."
                    )
                    company_data.loc[:, "normalized_value"] = company_data[
                        "value"
                    ]  # No normalization applied

                # Plot normalized lines
                fig.add_trace(
                    go.Scatter(
                        x=company_data["quarter"],
                        y=company_data["normalized_value"],
                        mode="lines",
                        name=f"{company} (normalized)",
                        hoverinfo="x+y",
                    )
                )
            else:
                # Plot regular lines
                fig.add_trace(
                    go.Scatter(
                        x=company_data["quarter"],
                        y=company_data["value"],
                        mode="lines",
                        name=company,
                        hoverinfo="x+y",
                    )
                )

    if plot_type == "normalized":
        # Calculate the average of actual values for each quarter
        average_data = data.groupby("quarter")["value"].mean().reset_index()

        # Normalize the average value (just like individual company values)
        first_non_zero_avg = average_data["value"][average_data["value"] != 0].iloc[0]
        average_data["normalized_value"] = average_data["value"] / first_non_zero_avg

        # Add the average normalized line to the plot
        fig.add_trace(
            go.Scatter(
                x=average_data["quarter"],
                y=average_data["normalized_value"],
                mode="lines",
                name="Average (normalized)",
                line=dict(dash="dash", color="black"),
                hoverinfo="x+y",
            )
        )

        y_axis_title = "Normalized Value (Proportional to 1)"
    else:
        # Calculate the average value for each quarter
        average_data = data.groupby("quarter")["value"].mean().reset_index()

        # Add the average line to the plot
        fig.add_trace(
            go.Scatter(
                x=average_data["quarter"],
                y=average_data["value"],
                mode="lines",
                name="Average",
                line=dict(dash="dash", color="black"),
                hoverinfo="x+y",
            )
        )

        y_axis_title = description

    # Update layout
    fig.update_layout(
        title=f"Comparison of {account} - {description}",
        xaxis_title="Quarter",
        yaxis_title=y_axis_title,
        template="plotly_white",
        showlegend=True,
    )

    # Show plot
    fig.show()

    return data

In [ ]:
# Example usage:
data = plot_company_comparison(
    df=df,  # Your DataFrame containing the financial data
    companies=["OI SA", "TELEFONICA BRASIL SA", "TIM SA"],
    account="01.01.01",  # Example account code for "Lucro do Período"
    start_date="2013-12-31",  # Filter from this date
    end_date="2020-12-31",  # Optional: Filter until this date
    plot_type="lines",  # Choose between "lines" and "normalized"
)
# Example usage:
data = plot_company_comparison(
    df=df,  # Your DataFrame containing the financial data
    companies=["OI SA", "TELEFONICA BRASIL SA", "TIM SA"],
    account="03.01",  # Example account code for "Lucro do Período"
    # start_date='2020-12-31',  # Filter from this date
    # end_date='2023-12-31',     # Optional: Filter until this date
    plot_type="normalized",  # Choose between "lines" and "normalized"
)

### World Data

In [ ]:
serie = "1"

url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{serie}/dados?formato=json"
response = requests.get(url)
data = response.json()
data = pd.DataFrame(data)

In [54]:
def get_median_data(df):
    # Remover a informação de fuso horário (timezone) antes de converter para períodos
    df.index = df.index.tz_localize(None)

    # Criar uma coluna de trimestre ('Quarter')
    df["quarter"] = df.index.to_period("Q")

    # Calcular a mediana de 'Adj Close' por trimestre
    median = df.groupby("quarter")["value"].median()
    # mean = df.groupby('quarter')['Adj Close'].mean()

    # Convert last_dates to the actual last date of the quarter
    # Using the 'Q' period information from 'Quarter'
    last_quarter_dates = df["quarter"].map(lambda x: x.end_time)

    # Converter as datas para manter apenas a parte da data (sem tempo)
    last_quarter_dates = last_quarter_dates.dt.date.drop_duplicates()

    # Criar um DataFrame com a última data correta de cada trimestre e a mediana trimestral
    quarterly_summary = pd.DataFrame(
        {
            "quarter": last_quarter_dates.values,  # Drop duplicates to keep one last date per quarter
            "median": median.values,
        }
    )

    return quarterly_summary

In [4]:
comm = [
    "ELETROMIDIA SA",
    "ALGAR TELECOM SA",
    "BRISANET PARTICIPACOES SA",
    "DESKTOP SA",
    "GIGA MAIS FIBRA TELECOMUNICACOES SA",
    "OI SA",
    "TELEC BRASILEIRAS SA TELEBRAS",
    "TELEFONICA BRASIL SA",
    "TIM SA",
    "UNIFIQUE TELECOMUNICACOES SA",
]

In [17]:
# sector = 'COMUNICACOES'
db_file = f"..\data\{settings.db_filepath}"

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Fetch all table names excluding internal SQLite tables
cursor.execute(
    "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
)
tables = cursor.fetchall()

df = pd.read_sql_query(f"SELECT * FROM company_info", conn)
companies = df[df["ticker_codes"] != ""]
companies = companies[
    ["cvm_code", "company_name", "ticker", "ticker_codes"]
].drop_duplicates()

In [ ]:
# Get the unique ticker codes from the 'ticker_codes' column
ticker_codes = companies["ticker_codes"].unique()

# Split the comma-separated ticker codes, flatten the list, and append '.SA' if necessary
ticker_list = [
    f"{code.strip()}.SA" if not code.strip().endswith(".SA") else code.strip()
    for tickers in ticker_codes
    for code in tickers.split(",")
    if code
]

ticker_list

In [ ]:
# Get the unique ticker codes from the 'ticker_codes' column
ticker_codes = companies["ticker_codes"].unique()

# Split the comma-separated ticker codes, flatten the list, and append '.SA' if necessary
ticker_list = [
    f"{code.strip()}.SA" if not code.strip().endswith(".SA") else code.strip()
    for tickers in ticker_codes
    for code in tickers.split(",")
    if code
]

# Remove empty strings (if any) and display the result
companies = [code for code in ticker_list if code]
len(companies)

In [ ]:
# Baixar os dados
last_date = "1900-01-01"
# companies = ['AAPL', 'ITUB3.SA', 'ITUB4.SA', 'GOOG']
data = yf.download(" ".join(companies), start=last_date)

In [ ]:
historical_data = {}
companies2 = []
data_error = {}
start_time = time.time()
for i, company in enumerate(companies):
    try:
        # Selecionar os dados de company e remover NaNs
        df = data.xs(company, axis=1, level=1).dropna()
        df["value"] = df["Adj Close"]

        # get median data from df
        df = get_median_data(df)

        historical_data[company] = df
        companies2.append(company)
        extra_info = [company]
        print_info(i, extra_info, start_time, len(companies))
    except Exception as e:
        data_error[company] = company

In [ ]:
# Now find the overall min and max quarter across all tickers
all_quarters = [data["quarter"] for data in historical_data.values()]
max_quarter = pd.concat(all_quarters).max()
previous_quarter = pd.Period(max_quarter, freq="Q") - 1
last_date = previous_quarter.end_time.date()

data2 = yf.download(" ".join(companies2), start=last_date)

### Yahoo Data

In [5]:
def load_data(files):
    """
    Run financial data from the database and process it into DataFrames.

    Args:
        files (str): The name part of the database file to load.

    Returns:
        dict: A dictionary where keys are sectors and values are DataFrames containing the NSD data for that sector.
    """
    try:
        db_file = os.path.join(
            settings.data_folder, f"{settings.db_filepath.split('.')[0]} {files}.db"
        )
        db_file = "D:/Fausto Stangler/Documentos/Python/FLY/backend/data/b3 standard.db"
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Fetch all table names excluding internal SQLite tables
        cursor.execute(
            "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
        )
        tables = cursor.fetchall()

        dfs = {}
        total_lines = 0
        start_time = time.time()  # Initialize start time for progress tracking

        # Iterate through each table (sector) and process the data
        for i, table in enumerate(tables):
            try:
                sector = table[0]
                df = pd.read_sql_query(f"SELECT * FROM {sector}", conn)

                # Normalize date columns to datetime format
                df["quarter"] = pd.to_datetime(df["quarter"], errors="coerce")

                # Normalize numeric columns
                df["value"] = pd.to_numeric(df["value"], errors="coerce")

                # Fill missing 'value' with 0
                df["value"] = df["value"].fillna(0)

                # Identify rows where 'account' is missing or NaN and 'value' has been set to 0
                missing_account = df["account"].isna() | df["account"].str.strip().eq(
                    ""
                )
                df.loc[missing_account, "account"] = (
                    "0"  # Set 'account' to '0' (as text) for these rows
                )

                # Filter out only the latest versions for each group
                # df, _ = self.filter_newer_versions(df)
                dfs[sector] = df  # Store the DataFrame with the sector as the key
                total_lines += len(df)  # Update the total number of processed lines

                # Display progress
                extra_info = [
                    f"Loaded {len(df)} items from {sector} in {files}, total {total_lines}"
                ]
                print_info(
                    i, extra_info, start_time, len(tables)
                )  # Removed the total_files argument

                print("break")
                break

            except Exception as e:
                # system.log_error(f"Error processing table {table}: {e}")
                print(e)

        conn.close()
        return dfs

    except Exception as e:
        print(e)
        # system.log_error(f"Error loading existing financial statements: {e}")
        return {}

In [6]:
def load_company_data():

    company_data = {}

    try:
        db_file = os.path.join(settings.data_folder, f"{settings.db_filepath}")
        db_file = "D:/Fausto Stangler/Documentos/Python/FLY/backend/data/b3.db"
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        cursor.execute(f"SELECT * FROM {settings.company_table}")

        for row in cursor.fetchall():
            company_name = row[settings.company_columns.index("company_name")]
            company_data[company_name] = dict(zip(settings.company_columns, row))

    except Exception as e:
        # system.log_error(e)
        print(e)

    return company_data

In [11]:
def get_median_data(df):
    # Remover a informação de fuso horário (timezone) antes de converter para períodos
    df.index = df.index.tz_localize(None)

    # Criar uma coluna de trimestre ('Quarter')
    df["quarter"] = df.index.to_period("Q")

    # Calcular a mediana de 'Adj Close' por trimestre
    median = df.groupby("quarter")["value"].median()
    # mean = df.groupby('quarter')['Adj Close'].mean()

    # Convert last_dates to the actual last date of the quarter
    # Using the 'Q' period information from 'Quarter'
    last_quarter_dates = df["quarter"].map(lambda x: x.end_time)

    # Converter as datas para manter apenas a parte da data (sem tempo)
    last_quarter_dates = last_quarter_dates.dt.date.drop_duplicates()

    # Criar um DataFrame com a última data correta de cada trimestre e a mediana trimestral
    quarterly_summary = pd.DataFrame(
        {
            "quarter": last_quarter_dates.values,  # Drop duplicates to keep one last date per quarter
            "median": median.values,
        }
    )

    return quarterly_summary

In [7]:
statements_data = load_data(settings.statements_standard)
df_statements = statements_data["COMUNICACOES"]

7.14% 1+13, 0.480669s per item, Time: 0h 00m 00s + 0h 00m 06s Loaded 90669 items from COMUNICACOES in standard, total 90669
break


In [8]:
all_companies = load_company_data()
df_companies = pd.DataFrame(all_companies).T.reset_index(drop=True)

In [9]:
# merge statements and company info

columns = ["company_name"]
cols_to_merge = columns + [
    "cvm_code",
    "ticker",
    "ticker_codes",
    "isin_codes",
    "listing",
]
# Merge the two DataFrames on the 'company_name' column
df_statements_companies = pd.merge(
    df_statements, df_companies[cols_to_merge], on=columns, how="left"
)

list_of_tickers = df_statements_companies[
    ["company_name", "ticker_codes"]
].drop_duplicates()
list_of_quarters = df_statements_companies[
    ["company_name", "ticker_codes", "quarter"]
].drop_duplicates()

In [12]:
# get historical data from yahoo finance

historical_data = {}
last_date = "1950-01-01"

# Iterate over each row and process tickers
for index, row in list_of_tickers.iterrows():
    company_name = row["company_name"]
    tickers = row["ticker_codes"]

    # Ensure that tickers are passed as a list by splitting any comma-separated string
    tickers = tickers.split(",") if isinstance(tickers, str) else tickers

    for ticker in tickers:

        if ticker:
            print(company_name, ticker)
            df = yf.download(
                ticker + ".SA", start=last_date, group_by="ticker", progress=False
            )
            df["value"] = df["Adj Close"]
            df = get_median_data(df)

        historical_data[ticker] = df
    # break

ELETROMIDIA SA ELMD3
BRISANET PARTICIPACOES SA BRIT3
DESKTOP SA DESK3
OI SA OIBR3
OI SA OIBR4
TELEC BRASILEIRAS SA TELEBRAS TELB3
TELEC BRASILEIRAS SA TELEBRAS TELB4
TELEFONICA BRASIL SA VIVT3
TIM SA TIMS3
UNIFIQUE TELECOMUNICACOES SA FIQE3


In [13]:
# create new historical data rows

new_rows = []
for i, row in list_of_quarters.iterrows():
    company_name = row["company_name"]
    quarter = row["quarter"]
    tickers = row["ticker_codes"]

    tickers = tickers.split(",") if isinstance(tickers, str) else tickers
    for ticker in tickers:
        if ticker:
            # Separate ticker into no digits and only digits
            tick = "".join(
                [char for char in ticker if not char.isdigit()]
            )  # Ticker without digits
            ticker_digit = "".join(
                [char for char in ticker if char.isdigit()]
            )  # Only digits

            mask = df_statements_companies["company_name"] == company_name
            mask &= df_statements_companies["quarter"] == quarter
            mask &= df_statements_companies["ticker"] == tick

            new_row = df_statements_companies[mask].iloc[0].copy()

            # Modify the necessary columns in the copied row
            new_row["type"] = "Cotações Históricas"
            new_row["frame"] = settings.tipos_acoes[ticker_digit]
            new_row["account"] = (
                f"99.{ticker_digit}"  # ticker_digit is assumed to be extracted from the original ticker
            )
            new_row["description"] = "Cotação Mediana do Trimestre"

            # Ensure 'quarter' in 'historical_data' is converted to datetime
            df_historical_data = historical_data.get(
                tick + ticker_digit, pd.DataFrame()
            )  # Use .get() to avoid KeyError if key doesn't exist
            if df_historical_data.empty:
                new_value = pd.NA
            else:
                df_historical_data["quarter"] = pd.to_datetime(
                    df_historical_data["quarter"]
                )

                # Filter the 'historical_data' for the correct 'quarter' and get the 'median' value
                mask = df_historical_data["quarter"] == pd.to_datetime(quarter)
                new_value = df_historical_data[mask]["median"].values

                # If no values are found, set to pd.NA
                if len(new_value) == 0:
                    new_value = pd.NA
                else:
                    new_value = new_value[0]  # Extract the actual median value

            # Set the new value in the new row
            new_row["value"] = new_value

            new_rows.append(new_row)
            print(company_name, quarter, tick, ticker_digit)

ELETROMIDIA SA 2019-12-31 00:00:00 ELMD 3
ELETROMIDIA SA 2020-03-31 00:00:00 ELMD 3
ELETROMIDIA SA 2020-06-30 00:00:00 ELMD 3
ELETROMIDIA SA 2020-09-30 00:00:00 ELMD 3
ELETROMIDIA SA 2020-12-31 00:00:00 ELMD 3
ELETROMIDIA SA 2021-03-31 00:00:00 ELMD 3
ELETROMIDIA SA 2021-06-30 00:00:00 ELMD 3
ELETROMIDIA SA 2021-09-30 00:00:00 ELMD 3
ELETROMIDIA SA 2021-12-31 00:00:00 ELMD 3
ELETROMIDIA SA 2022-03-31 00:00:00 ELMD 3
ELETROMIDIA SA 2022-06-30 00:00:00 ELMD 3
ELETROMIDIA SA 2022-09-30 00:00:00 ELMD 3
ELETROMIDIA SA 2022-12-31 00:00:00 ELMD 3
ELETROMIDIA SA 2023-03-31 00:00:00 ELMD 3
ELETROMIDIA SA 2023-06-30 00:00:00 ELMD 3
ELETROMIDIA SA 2023-09-30 00:00:00 ELMD 3
ELETROMIDIA SA 2023-12-31 00:00:00 ELMD 3
ELETROMIDIA SA 2024-03-31 00:00:00 ELMD 3
ELETROMIDIA SA 2024-06-30 00:00:00 ELMD 3
BRISANET PARTICIPACOES SA 2020-12-31 00:00:00 BRIT 3
BRISANET PARTICIPACOES SA 2021-03-31 00:00:00 BRIT 3
BRISANET PARTICIPACOES SA 2021-06-30 00:00:00 BRIT 3
BRISANET PARTICIPACOES SA 2021-09-30 00:00:

In [15]:
# insert new rows into df_statements_companies

df_final = pd.concat(
    [df_statements_companies, pd.DataFrame(new_rows)], ignore_index=True
).drop_duplicates()

mask = df_final["ticker"] == "ELMD"
# mask&= df_historical_statements_2['quarter'] == '2020-12-31'
mask &= df_final["type"] == "Cotações Históricas"

df_final

# sort and select only statement columns

,nsd,sector,subsector,segment,company_name,quarter,version,type,frame,account,description,value,cvm_code,ticker,ticker_codes,isin_codes,listing
0,99116,COMUNICACOES,MIDIA,PUBLICIDADE E PROPAGANDA,ELETROMIDIA SA,2019-12-31,1,DFs Consolidadas,Balanço Patrimonial Ativo,01,Ativo Total,2905300000.0,25569,ELMD,ELMD3,BRELMDACNOR3,CIA NOVO MERCADO
1,99116,COMUNICACOES,MIDIA,PUBLICIDADE E PROPAGANDA,ELETROMIDIA SA,2019-12-31,1,DFs Consolidadas,Balanço Patrimonial Ativo,01.01,Ativo Circulante de Curto Prazo,1061320000.0,25569,ELMD,ELMD3,BRELMDACNOR3,CIA NOVO MERCADO
2,99116,COMUNICACOES,MIDIA,PUBLICIDADE E PROPAGANDA,ELETROMIDIA SA,2019-12-31,1,DFs Consolidadas,Balanço Patrimonial Ativo,01.01.01,Caixa e Equivalentes de Caixa de Curto Prazo,380180000.0,25569,ELMD,ELMD3,BRELMDACNOR3,CIA NOVO MERCADO
3,99116,COMUNICACOES,MIDIA,PUBLICIDADE E PROPAGANDA,ELETROMIDIA SA,2019-12-31,1,DFs Consolidadas,Balanço Patrimonial Ativo,01.01.01.01,Caixa e Bancos de Curto Prazo,130980000.0,25569,ELMD,ELMD3,BRELMDACNOR3,CIA NOVO MERCADO
4,99116,COMUNICACOES,MIDIA,PUBLICIDADE E PROPAGANDA,ELETROMIDIA SA,2019-12-31,1,DFs Consolidadas,Balanço Patrimonial Ativo,01.01.01.02,Aplicações Líquidas de Curto Prazo,249200000.0,25569,ELMD,ELMD3,BRELMDACNOR3,CIA NOVO MERCADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91026,129891,COMUNICACOES,TELECOMUNICACOES,TELECOMUNICACOES,UNIFIQUE TELECOMUNICACOES SA,2023-06-30,1,Cotações Históricas,Ações Ordinárias (ON),99.3,Cotação Mediana do Trimestre,3.278778,26050,FIQE,FIQE3,BRFIQEACNOR5,CIA NOVO MERCADO
91027,131935,COMUNICACOES,TELECOMUNICACOES,TELECOMUNICACOES,UNIFIQUE TELECOMUNICACOES SA,2023-09-30,1,Cotações Históricas,Ações Ordinárias (ON),99.3,Cotação Mediana do Trimestre,3.719842,26050,FIQE,FIQE3,BRFIQEACNOR5,CIA NOVO MERCADO
91028,134906,COMUNICACOES,TELECOMUNICACOES,TELECOMUNICACOES,UNIFIQUE TELECOMUNICACOES SA,2023-12-31,1,Cotações Históricas,Ações Ordinárias (ON),99.3,Cotação Mediana do Trimestre,3.490817,26050,FIQE,FIQE3,BRFIQEACNOR5,CIA NOVO MERCADO
91029,137478,COMUNICACOES,TELECOMUNICACOES,TELECOMUNICACOES,UNIFIQUE TELECOMUNICACOES SA,2024-03-31,1,Cotações Históricas,Ações Ordinárias (ON),99.3,Cotação Mediana do Trimestre,3.726683,26050,FIQE,FIQE3,BRFIQEACNOR5,CIA NOVO MERCADO


### Finantial Ratios

In [14]:
sector = "COMUNICACOES"
db_file = f"..\data\{settings.db_filepath.split('.')[0]} {settings.markets_file}.db"

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Fetch all table names excluding internal SQLite tables
cursor.execute(
    "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
)
tables = cursor.fetchall()

df = pd.read_sql_query(f"SELECT * FROM {sector}", conn)
# df['account_description'] = df['account'] + ' - ' + df['description']
df["quarter"] = pd.to_datetime(df["quarter"])
# Store unique values

In [15]:
uv = {}
for col in df.columns:
    if col not in ["nsd", "value"]:
        uv[col] = sorted(df[col].unique().tolist())
uv

{'sector': ['COMUNICACOES'],
 'subsector': ['MIDIA', 'TELECOMUNICACOES'],
 'segment': ['PUBLICIDADE E PROPAGANDA', 'TELECOMUNICACOES'],
 'company_name': ['ALGAR TELECOM SA',
  'BRISANET PARTICIPACOES SA',
  'DESKTOP SA',
  'ELETROMIDIA SA',
  'GIGA MAIS FIBRA TELECOMUNICACOES SA',
  'OI SA',
  'TELEC BRASILEIRAS SA TELEBRAS',
  'TELEFONICA BRASIL SA',
  'TIM SA',
  'UNIFIQUE TELECOMUNICACOES SA'],
 'quarter': [Timestamp('2010-12-31 00:00:00'),
  Timestamp('2011-03-31 00:00:00'),
  Timestamp('2011-06-30 00:00:00'),
  Timestamp('2011-09-30 00:00:00'),
  Timestamp('2011-12-31 00:00:00'),
  Timestamp('2012-03-31 00:00:00'),
  Timestamp('2012-06-30 00:00:00'),
  Timestamp('2012-09-30 00:00:00'),
  Timestamp('2012-12-31 00:00:00'),
  Timestamp('2013-03-31 00:00:00'),
  Timestamp('2013-06-30 00:00:00'),
  Timestamp('2013-09-30 00:00:00'),
  Timestamp('2013-12-31 00:00:00'),
  Timestamp('2014-03-31 00:00:00'),
  Timestamp('2014-06-30 00:00:00'),
  Timestamp('2014-09-30 00:00:00'),
  Timestamp(

### Bovespa Data

In [19]:
def parse_company_data(url, base_url="https://bvmf.bmfbovespa.com.br/sig/"):
    response = requests.get(url, headers=header_random(), verify=False)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # Find the relevant table
        table = soup.find("table", {"width": "95%"})

        # Initialize lists to store data
        company_name = []
        trading_name = []
        listing = []
        ticker = []
        # url_mercado = []

        if table:
            # Iterate through rows of the table and extract data
            for row in table.find_all("tr")[1:]:  # Skip the header
                columns = row.find_all("td")

                # Check if the row has enough columns
                if len(columns) >= 4:
                    company_name.append(
                        clean_text(columns[0].text.strip())
                    )  # Razão Social
                    trading_name.append(
                        clean_text(columns[1].text.strip())
                    )  # Nome de Pregão
                    listing.append(
                        clean_text(columns[2].text.strip())
                    )  # Tipo de Mercado
                    ticker.append(clean_text(columns[3].text.strip()))  # Sigla

                    # # Extract the link
                    # link = columns[0].find('a')['href']
                    # full_link = f"{base_url}{link}"
                    # url_mercado.append(full_link)

        # Return the extracted data as a DataFrame
        df = pd.DataFrame(
            {
                "company_name": company_name,
                "trading_name": trading_name,
                "ticker": ticker,
                "listing": listing,
                # 'url_mercado': url_mercado
            }
        )
        df["listing"] = (
            df["listing"].map(settings.governance_levels).fillna(df["listing"])
        )

    else:
        df = pd.DataFrame()

    return df

In [20]:
def get_companies_listing():
    try:
        from requests.packages.urllib3.exceptions import InsecureRequestWarning

        # Ignorar avisos de SSL (opcional)
        requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

        # List of characters A-Z and 0-9
        chars = list(string.digits) + list(string.ascii_uppercase)

        # Generate URLs based on each character
        base_urls = [
            f"https://bvmf.bmfbovespa.com.br/sig/FormConsultaEmpResultado.asp?strLetraInicial={char}"
            for char in chars
        ]

        # Initialize an empty DataFrame to store all companies
        dfs = []

        # Iterate through each URL and fetch the data
        start_time = time.time()
        for i, url in enumerate(base_urls):
            # companies from url
            df = parse_company_data(url)
            dfs.append(df)

            extra_info = [url, f"{len(df)} companies listed"]
            print_info(i, extra_info, start_time, len(base_urls))

        # DataFrame with all companies
        companies = pd.concat(dfs, ignore_index=True)

    except requests.exceptions.RequestException as e:
        companies = pd.DataFrame()

    return companies

In [21]:
def fetch_stock_info(soup):
    stock_info = {}

    # Locate the rows with "Nome da Ação" to get stock name and ticker
    stock_rows = soup.find_all("td", class_="tittabela", colspan="7")

    for row in stock_rows:
        stock_text = row.get_text(strip=True)
        if "Nome da Ação" in stock_text:
            # Extract the stock name and ticker (in parentheses)
            stock_name = stock_text.split("(")[0].replace("Nome da Ação:", "").strip()
            stock_ticker = stock_text.split("(")[1].replace(")", "").strip()

            # Add stock_name and stock_ticker to the dictionary
            stock_info[stock_name] = stock_ticker
    return stock_info

In [22]:
def get_pages_to_download(companies, start_date):
    tipo = "RES_MERC_VISTA"

    date_range = [
        date.strftime("%m-%Y")
        for date in pd.date_range(
            start=start_date,
            end=pd.Timestamp.today() - pd.offsets.MonthEnd(1),
            freq="MS",
        )
    ]

    # Generate the items list with ticker, company_name, trading_name, and date
    items = [
        [row["ticker"], row["company_name"], row["trading_name"], mes]
        for _, row in companies.iterrows()
        for mes in date_range
    ]

    return items

In [23]:
# Helper function to convert text with '.' as thousand separator and ',' as decimal separator
def convert_to_float(value):
    try:
        # Remove thousand separator and replace decimal separator
        return float(value.replace(".", "").replace(",", "."))
    except ValueError:
        return None


def convert_to_int(value):
    try:
        # Remove thousand separator and convert to int
        return int(value.replace(".", "").replace(",", ""))
    except ValueError:
        return None


def fetch_trades(soup, stock_info, ticker_code, company_name, trading_name, date):
    data = []

    # Find all tables that contain daily trading information
    tables = soup.find_all("table")

    for table in tables:
        rows = table.find_all("tr")

        for row in rows:
            columns = row.find_all("td")
            if len(columns) == 12:  # Ensure the row has the expected number of columns
                dia = columns[0].get_text(strip=True)
                especif = columns[1].get_text(strip=True)

                # Convert trades, quantity, and volume to int
                trades = convert_to_int(columns[2].get_text(strip=True))
                quantity = convert_to_int(columns[4].get_text(strip=True))
                volume = convert_to_int(columns[5].get_text(strip=True))

                # Convert open, low, high, average, close to float
                open = convert_to_float(columns[7].get_text(strip=True))
                low = convert_to_float(columns[8].get_text(strip=True))
                high = convert_to_float(columns[9].get_text(strip=True))
                average = convert_to_float(columns[10].get_text(strip=True))
                close = convert_to_float(columns[11].get_text(strip=True))

                # Filter out unwanted rows based on some patterns
                if "Negociações" in dia or "Dia" in dia or "Total" in dia:
                    continue  # Skip unwanted rows

                # Combine 'dia' with 'mes' to create a full date
                full_date = pd.to_datetime(f"{dia}-{date}", format="%d-%m-%Y")

                # Split 'Especif.' into stock code and listing
                parts = especif.split()

                # Map 'ticker_code' from the first part (stock_name) using stock_info
                ticker_ = parts[0]
                ticker = stock_info.get(
                    ticker_, ticker_
                )  # Map or fallback to stock_code
                ticker_number = "".join([char for char in ticker if char.isdigit()])

                # Extract 'listing' if available
                listing = parts[1] if len(parts) > 1 else None

                if listing == "*":
                    trades = (
                        trades * 10
                    )  # (transformar lotes de mil em 10 lotes de 100)
                    listing = ""

                # Map 'listing' using the governance_levels dictionary
                if listing in settings.governance_levels:
                    listing = settings.governance_levels[listing]

                # Append row data to the list
                data.append(
                    {
                        "company_name": company_name,
                        "trading_name": trading_name,
                        "ticker_code": ticker_code + ticker_number,
                        "date": full_date,
                        "listing": listing,
                        "trades": trades,
                        "quantity": quantity,
                        "volume": volume,
                        "open": open,
                        "low": low,
                        "high": high,
                        "average": average,
                        "close": close,
                    }
                )

    df = pd.DataFrame(data).drop_duplicates()
    return df

In [ ]:
settings.db_filepath

In [25]:
db_file = f"..\data\{settings.db_filepath}"

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

In [ ]:
companies.info()

In [ ]:
# Generate the date range from January 1996 to the current month
start_date = "1991-01"
# start_date = '1986-01'
start_date = "1997-01"  # temp debug


companies = get_companies_listing()
pages = get_pages_to_download(companies, start_date)
error_url = []

In [ ]:
tipo = "RES_MERC_VISTA"

dfs = []

start_time = time.time()
for i, (ticker_code, company_name, trading_name, date) in enumerate(items):
    if [ticker_code, date] not in error_url:

        # debug skip
        if "BRSR" not in ticker_code:
            continue

        # Fetch the webpage content
        url = f"https://bvmf.bmfbovespa.com.br/sig/FormConsultaMercVista.asp?strTipoResumo={tipo}&strSocEmissora={ticker_code}&strDtReferencia={date}&strIdioma=P&intCodNivel=2&intCodCtrl=160"
        try:
            # get page response
            response = requests.get(url, headers=header_random(), verify=False)

            if response.status_code == 200:
                # Parse the HTML using BeautifulSoup
                soup = BeautifulSoup(response.text, "html.parser")

                # Check if page has target content
                stock_info = fetch_stock_info(soup)

                # if page has target content, get data
                if stock_info:
                    df = fetch_trades(
                        soup, stock_info, ticker_code, company_name, trading_name, date
                    )
                    dfs.append(df)

                    # print status
                    extra_info = [ticker_code, date, url]
                    print_info(i, extra_info, start_time, len(items))
                    if i == 129499:
                        break

                # if not target content, add to error list
                else:
                    error_url.append([ticker_code, date])

                    # print status
                    extra_info = [ticker_code, date]
                    print_info(i, extra_info, start_time, len(items))

            # if response error
            else:
                print(
                    f"Failed to fetch the page. Status code: {response.status_code}, {i}"
                )

        except requests.exceptions.RequestException as e:
            print(f"Error requesting {ticker_code}, {date}: {e}")

    # if already checked item for error
    else:
        print(ticker_code, date)
        pass

In [ ]:
pd.concat(dfs)

In [ ]:
import requests
from bs4 import BeautifulSoup

# Example variables
tipo = "RES_MERC_VISTA"
ticker = "BRSR"  # Replace this with a value from companies['ticker']
mes = "04/2017"

# Build the base URL
base_url = f"https://bvmf.bmfbovespa.com.br/sig/FormConsultaMercVista.asp?strTipoResumo={tipo}&strSocEmissora={ticker}&strDtReferencia={mes}&strIdioma=P&intCodNivel=2&intCodCtrl=160"

# Fetch the webpage content
try:
    response = requests.get(base_url, headers=header_random(), verify=False)
    if response.status_code == 200:
        # Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")
        print(f"URL: {base_url} - Status: {response.status_code} - Success")

        # Locate the tables by finding the table with id "tblResDiario"
        tables = soup.find_all("table", {"id": "tblResDiario"})

    else:
        print(f"Failed to fetch the page. Status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Error requesting {base_url}: {e}")

In [ ]:
if 1 == 1:
    stock_info = {}

    # Locate the rows with "Nome da Ação" to get stock name and ticker
    stock_rows = soup.find_all("td", class_="tittabela", colspan="7")

    for row in stock_rows:
        stock_text = row.get_text(strip=True)
        if "Nome da Ação" in stock_text:
            # Extract the stock name and ticker (in parentheses)
            stock_name = stock_text.split("(")[0].replace("Nome da Ação:", "").strip()
            stock_ticker = stock_text.split("(")[1].replace(")", "").strip()

            # Add stock_name and stock_ticker to the dictionary
            stock_info[stock_name] = stock_ticker

stock_info

In [ ]:
# Parsing the table data
if 1 == 1:

# Create a DataFrame and remove duplicates
df = pd.DataFrame(data).drop_duplicates()

# Display the DataFrame
df.iloc[:60]
